# eq_IO_Taux_bias: Zonal structure of time-mean Zonal Mean Stress (Taux) in the equatorial Indian Ocean

Computes the zonal root mean square error (RMSE) of equatorial Indian Ocean (50°E-110°E) climatological (time and meridional 10°S-10°N average) zonal wind stress (Taux) between model and observations

## Datasets Used for Zonal Mean Stress Comparison

The following reference reanalysis datasets are used to evaluate the model simulations:

- **TropFlux (1979-2018)**: The primary dataset for comparison. TropFlux provides estimates of air-sea heat, momentum, and freshwater fluxes over the tropical oceans, designed specifically for tropical climate analysis by combining satellite data and in-situ measurements.

Additional reanalysis datasets used for comparison include:
- **20CRv2 (1871-2012)**: The 20th Century Reanalysis provides global atmospheric data based on surface pressure observations and historical data assimilation, offering a long-term record of climate and weather variability spanning over 140 years.
- **ERA-Interim (1979-2018)**: The European Centre for Medium-Range Weather Forecasts (ECMWF) reanalysis dataset delivers high-resolution atmospheric and surface variables using modern assimilation techniques, often used for climate and weather analysis.
- **NCEP2 (1979-2018)**: The second-generation reanalysis from the National Centers for Environmental Prediction (NCEP), offering improved global atmospheric and surface data by correcting known issues from the earlier version and assimilating more recent observations.

## Dask Cluster 

Start a cluster with multiple cores 

In [3]:
from dask.distributed import Client, progress
client = Client()
client

/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.4/lib/python3.11/site-packages/distributed/node.py:168: ResourceWarning: unclosed <socket.socket fd=157, family=2, type=1, proto=6, laddr=('0.0.0.0', 0)>
  if retries_left < 1:
/g/data/xp65/public/apps/med_conda/envs/esmvaltool-0.4/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38453 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/38453/status,
Dashboard: /proxy/38453/status,Workers: 7
Total threads: 7,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35263,Workers: 7
Dashboard: /proxy/38453/status,Total threads: 7
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:42441,Total threads: 1
Dashboard: /proxy/33347/status,Memory: 4.57 GiB
Nanny: tcp://127.0.0.1:33453,


## Datasets

In [4]:
from esmvalcore.dataset import Dataset
from esmvalcore.config import CFG
CFG['rootpath']['OBS6'].append(r'/g/data/kj13/datasets/esmvaltool/obsdata-v2') #tier3

In [9]:
model_datasets = {
"ACCESS-ESM1-5": 
    Dataset(
    short_name='tauu',
    project='CMIP6',
    mip="Amon",
    exp="historical",
    ensemble="r1i1p1f1",
    timerange="19790101/20190101",
    dataset="ACCESS-ESM1-5",
    grid="gn"
)}

obs_datasets = {
 "ERA5": 
     Dataset(
     short_name='tauu',
     dataset='ERA5',
     mip="Amon",
     project='OBS6',
     type='reanaly',
     timerange="19790101/20190101",
     tier=3),
# "NOAA-CIRES-20CR-V2": 
#     Dataset(
#     short_name='tauu',
#     dataset='NOAA-CIRES-20CR-V2',
#     mip="Amon",
#     project='OBS6',
#     type='reanaly',
#     timerange="187101/201212",
#     tier=2),
}



## Add Ancillary files

In [10]:
for name, dataset in model_datasets.items():
    dataset.add_supplementary(short_name='sftlf', mip="fx")
    dataset.add_supplementary(short_name="areacella", mip="fx")

In [11]:
model_datasets = {name: dataset.load() for name, dataset in model_datasets.items()}
obs_datasets = {name: dataset.load() for name, dataset in obs_datasets.items()}

 tauu: attribute positive not present
loaded from file /g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/Amon/tauu/gn/v20191115/tauu_Amon_ACCESS-ESM1-5_historical_r1i1p1f1_gn_185001-201412.nc
 tauu: attribute positive not present
loaded from file 
(for file /g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/fx/sftlf/gn/v20191115/sftlf_fx_ACCESS-ESM1-5_historical_r1i1p1f1_gn.nc)


InputFilesNotFound: No files were found for Dataset:
{'dataset': 'ERA5',
 'project': 'OBS6',
 'mip': 'Amon',
 'short_name': 'tauu',
 'frequency': 'mon',
 'long_name': 'Surface Downward Eastward Wind Stress',
 'modeling_realm': ['atmos'],
 'original_short_name': 'tauu',
 'standard_name': 'surface_downward_eastward_stress',
 'tier': 3,
 'timerange': '19790101/20190101',
 'type': 'reanaly',
 'units': 'Pa'}
session: 'session-5b38f8d5-39c5-4853-b2d4-5b329eca6e60_20250325_051525'
locally using glob patterns:
/g/data/ct11/access-nri/era5-derived/Tier3/ERA5/OBS6_ERA5_reanaly_*_Amon_tauu[_.]*nc
/g/data/ct11/access-nri/replicas/esmvaltool/obsdata-v2/Tier3/ERA5/OBS6_ERA5_reanaly_*_Amon_tauu[_.]*nc
/g/data/kj13/datasets/esmvaltool/obsdata-v2/Tier3/ERA5/OBS6_ERA5_reanaly_*_Amon_tauu[_.]*nc
/g/data/kj13/datasets/esmvaltool/obsdata-v2/Tier3/ERA5/OBS6_ERA5_reanaly_*_Amon_tauu[_.]*nc
/g/data/kj13/datasets/esmvaltool/obsdata-v2/Tier3/ERA5/OBS6_ERA5_reanaly_*_Amon_tauu[_.]*nc
/g/data/nf33/public/data/ESM

In [ ]:
from esmvalcore.preprocessor import annual_statistics
from esmvalcore.preprocessor import climate_statistics
from esmvalcore.preprocessor import convert_units
from esmvalcore.preprocessor import extract_region
from esmvalcore.preprocessor import extract_time
from esmvalcore.preprocessor import regrid
from esmvalcore.preprocessor import axis_statistics
from esmvalcore.preprocessor import mask_landsea

In [ ]:
def prepoc1(cube):
    nino3_ep_region = {"start_longitude": 50., "end_longitude": 110., "start_latitude": -10., "end_latitude": 10.}
    cube = extract_time(cube, start_year=1979, start_month=1, start_day=1, end_year=2014, end_month=12, end_day=31)        
    cube = mask_landsea(cube, mask_out="land")
    cube = climate_statistics(cube, operator="mean", period="full")
    cube = regrid(cube, target_grid="1x1", scheme="linear")
    cube = extract_region(cube, **nino3_ep_region)
    cube = axis_statistics(cube, operator="mean", axis="y")
    cube = convert_units(cube, units="1e-3 N/m2")
    return cube

def prepoc2(cube):
    nino3_latext_region = {"start_longitude": 50., "end_longitude": 110., "start_latitude": -25., "end_latitude": 25.}
    cube = extract_time(cube, start_year=1979, start_month=1, start_day=1, end_year=2014, end_month=12, end_day=31)        
    cube = mask_landsea(cube, mask_out="land")
    cube = climate_statistics(cube, operator="mean", period="full")
    cube = regrid(cube, target_grid="1x1", scheme="linear")
    cube = extract_region(cube, **nino3_latext_region)
    cube = convert_units(cube, units="1e-3 N/m2")
    return cube

In [ ]:
model_datasets_prep1 = {name: prepoc1(dataset) for name, dataset in model_datasets.items()}
obs_datasets_prep1 = {name: prepoc1(dataset) for name, dataset in obs_datasets.items()}

model_datasets_prep2 = {name: prepoc2(dataset) for name, dataset in model_datasets.items()}
obs_datasets_prep2 = {name: prepoc2(dataset) for name, dataset in obs_datasets.items()}

# Diagnostics Level 1


In [ ]:
import iris
import iris.quickplot as qplt
import matplotlib.pyplot as plt
import numpy as np

# Function to format x-axis labels with degree symbols and N/S
def format_longitude(x, pos):
    if x > 180:
        return f'{int(360 - x)}°W'
    elif x == 180:
        return f'{int(x)}°'
    else:
        return f'{int(x)}°E'

plt.figure(figsize=(10, 6))

obs_data = obs_datasets_prep1["ERA-Interim"]
model_data = model_datasets_prep1["ACCESS-ESM1-5"]

# Plot observation data in black
qplt.plot(obs_data, color='black', label='Observation', linewidth=4)

# Plot model data in blue
qplt.plot(model_data, color='blue', label='Model', linewidth=4)

rmse = np.sqrt(np.mean((obs_data.data - model_data.data) ** 2))

# Adding labels and title
plt.xlabel('Longitude')
plt.ylabel('Taux (1e-3 N/m2)')
plt.title('Mean Taux')
plt.grid(linestyle='--')
plt.yticks(np.arange(-60,21, step=20))
plt.axhline(y=0, color='black')

# Adding RMSE text on top of the plot
plt.text(0.5, 0.95, f'RMSE: {rmse:.2f} 1e-3 N/m2', fontsize=12, ha='center', transform=plt.gca().transAxes,
         bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

# Setting custom x-axis formatter
plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(format_longitude))

# Adding a legend
plt.legend()

# Display the plot
plt.show()

Figure 1: Zonal bias in the structure of zonal wind stress (Taux) across the equatorial Pacific (averaged between 5°S–5°N), primarily highlighting the trade winds bias (typically weaker circulation in the central Pacific and stronger in the western Pacific). The black curve represents the reference data, and the blue curve represents the model data. The metric displayed is the zonal RMSE between the model and reference curves.

# Diagnostics Level 2


In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Importing numpy for linear space creation
import numpy as np

import iris
import iris.plot as iplt

obs_data = obs_datasets_prep1["ERA-Interim"]
model_data = model_datasets_prep1["ACCESS-ESM1-5"]

# Use the coolwarm colormap
cmap = plt.get_cmap('coolwarm')

# Create a large figure
fig = plt.figure(figsize=(20, 10))  # Adjusted for full width

# Center the map on the Pacific Ocean (roughly around 180° longitude)
proj = ccrs.Orthographic(central_longitude=50, central_latitude=0)

# Define the normalization for the color scale with range of -100 to 100
norm = mcolors.Normalize(vmin=-100, vmax=100)

# Plot #1: contourf with axes longitude from -180 to 180
ax1 = plt.subplot(121, projection=proj)
cf1 = iplt.contourf(obs_data, cmap=cmap, norm=norm)
ax1.add_feature(cfeature.LAND, facecolor='gray')  # Add land feature with gray color
ax1.coastlines()
ax1.set_extent([50, 110, -25, 25], crs=ccrs.PlateCarree())
ax1.set_title("ref: ERA-Interim")

# Add gridlines for latitude and longitude
gl1 = ax1.gridlines(draw_labels=True, linestyle='--')
gl1.top_labels = False
gl1.right_labels = False

# Plot #2: contourf with axes longitude from 0 to 360
ax2 = plt.subplot(122, projection=proj)
cf2 = iplt.contourf(model_data, cmap=cmap, norm=norm)
ax2.add_feature(cfeature.LAND, facecolor='gray')  # Add land feature with gray color
ax2.coastlines()
ax2.set_extent([50, 110, -25, 25], crs=ccrs.PlateCarree())
ax2.set_title("ACCESS-ESM1-5 r1i1p1f1")

# Add gridlines for latitude and longitude
gl2 = ax2.gridlines(draw_labels=True, linestyle='--')
gl2.top_labels = False
gl2.right_labels = False

# Add a single colorbar at the bottom
# NOTE: The colorbar will be based on cf2, which already has the correct normalization applied.
cbar = fig.colorbar(cf2, ax=[ax1, ax2], orientation='horizontal', pad=0.15, shrink=0.8)
cbar.set_label('SST (°C)')
# cbar.set_clim(-100, 100)
# cbar.set_ticks(np.linspace(-100, 100, num=11))  # Set ticks ranging from -100 to 100

plt.subplots_adjust(bottom=0.3)  # Further adjust bottom margin to fit the colorbar
plt.show()


Figure 2: Time-mean zonal wind stress (Taux) bias in the equatorial Pacific, illustrating the southeasterly and northeasterly trade winds that form the surface branch of the Walker circulation. The left map shows the reference data, while the right map displays the model data.